In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W37`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W38`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W39`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W40`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W41`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W42`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W43`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W44`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [8]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
52031,2020-11-22 12:13:34+00:00,2020-11-22 12:13:40+00:00,Production,0049 of 0054,A2,None,None,KL2,Monday Assembly,Sunday Morning,iXUsr_Normie(7460),4,0.10,2020-48
52032,2020-11-22 06:43:57+00:00,2020-11-22 06:48:42+00:00,Production,0011 of 0015,T2,None,None,KL7,Monday Assembly,Sunday Morning,iXUsr_Normie(7460),3,4.75,2020-48
52040,2020-11-22 10:48:56+00:00,2020-11-22 10:52:11+00:00,Production,0033 of 0054,A2,None,None,KL2,Monday Assembly,Sunday Morning,iXUsr_Normie(7460),4,3.25,2020-48
63984,2020-11-19 16:09:04+00:00,2020-11-19 16:13:20+00:00,Production,0027 of 0029,E4,None,None,KL1,Friday Assembly,Thursday Afternoon,iXUsr_Tafai(10143),1,4.27,2020-48
82170,2020-11-20 18:52:41+00:00,2020-11-20 19:02:02+00:00,Preparation/Changeover,None,T2,None,None,KL7,None,Friday Afternoon,iXUsr_Kim Jie Yap 7055;,2,9.35,2020-48


In [5]:
data_in_use.shape

(94253, 14)